In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('run-data.xlsx')

In [3]:
data.shape

(16, 28)

In [4]:
# data.drop(len(data)-1, inplace=True)

In [5]:
# data.to_excel('../run-data.xlsx', index=False)

In [6]:
data.head()

,rollout,waitingTime,time,rew,E_num_train_rollouts,E_rollout_length,E_eval_freq,E_eval_num_eps,E_max_ep_steps,E_generation_ep_steps,...,A_clip_grads,A_gradient_clip,A_value_loss_coef,O_num_agents,O_rule_set,O_rule_set_params,O_environment,O_NS_mult,O_EW_mult,O_phase_end_offset
0,208,1861.870833,9.890320,413.011042,200,1024,48,30,900,750,...,True,1,1,8,NaN,{'cycle_length': 20},four_intersections,1.5,1,100
1,208,1861.508333,10.057287,414.745000,200,1024,48,30,900,750,...,True,1,1,8,NaN,{'cycle_length': 20},four_intersections,1.5,1,100
2,408,1840.322222,16.347096,403.694375,400,1024,48,30,900,750,...,True,1,1,8,NaN,{'cycle_length': 20},four_intersections,1.5,1,100
3,408,1843.663889,16.367913,407.831875,400,1024,48,30,900,750,...,True,1,1,8,NaN,{'cycle_length': 20},four_intersections,1.5,1,100
4,144,1859.706944,17.373463,416.635417,200,2048,48,30,900,750,...,True,1,1,8,NaN,{'cycle_length': 20},four_intersections,1.5,1,100


In [32]:
data[data['waitingTime'] < 1000]`

,rollout,waitingTime,time,rew,E_num_train_rollouts,E_rollout_length,E_eval_freq,E_eval_num_eps,E_max_ep_steps,E_test_num_eps,...,A_optimization_epochs,A_ppo_ratio_clip,A_discount,A_learning_rate,A_clip_grads,A_gradient_clip,A_value_loss_coef,O_num_agents,O_rule_set,O_rule_set_params
30,258,994.88,4.290420,645.457778,250,1024,98,25,1000,250,...,5,0.3,0.99,0.00001,True,1,1.0,8,NaN,{'length': 5}
118,251,998.33,5.760454,641.853333,250,1024,98,25,1000,250,...,15,0.3,0.99,0.00001,False,1,1.0,8,NaN,{'length': 5}


# Notes
- rollout_length: at least 1024 (test higher)
- gae_tau: at most 0.9 (maybe lower)
- entropy_weight: at least 0.1 (test higher)
- minibatch_size: at least 128 (maybe higher)
- optimization_epochs: 5 but 10 is very close
- ppo_ratio_clip: 0.3 but 0.2 is close
- clip_grads: true
- value_loss_coef: 1.0


- NS_mult: 1.0 or 1.5
- phase_end_offset: any (0 - 100)
- multi intersection cycle length: 20
- (best combo 1.5, 100, 20)

In [8]:
data[data['E_num_train_rollouts'] == 200]['waitingTime'].mean()

2115.3046874999995

In [9]:
data[data['E_num_train_rollouts'] == 400]['waitingTime'].mean()

1895.862326388889

In [19]:
data[data['O_phase_end_offset'] == 100]['waitingTime'].mean()

1768.1567429193901

In [24]:
list(data['O_rule_set_params'])

["{'cycle_length': 20}",
 "{'cycle_length': 30}",
 "{'cycle_length': 40}",
 "{'cycle_length': 20, 'phases': [{'duration': 0.25, 'probs': {'northLeft': {'gen': 0.01, 'rem': 0.27777777777777773}, 'northRight': {'gen': 0.01, 'rem': 0.27777777777777773}, 'southLeft': {'gen': 0.2, 'rem': 0.05555555555555555}, 'southRight': {'gen': 0.2, 'rem': 0.05555555555555555}, 'westTop': {'gen': 0.01, 'rem': 0.1111111111111111}, 'eastTop': {'gen': 0.01, 'rem': 0.1111111111111111}, 'westBottom': {'gen': 0.05, 'rem': 0.05555555555555555}, 'eastBottom': {'gen': 0.05, 'rem': 0.05555555555555555}}}, {'duration': 0.5, 'probs': {'northLeft': {'gen': 0.1, 'rem': 0.12500000000000003}, 'northRight': {'gen': 0.1, 'rem': 0.12500000000000003}, 'southLeft': {'gen': 0.1, 'rem': 0.12500000000000003}, 'southRight': {'gen': 0.1, 'rem': 0.12500000000000003}, 'westTop': {'gen': 0.1, 'rem': 0.12500000000000003}, 'eastTop': {'gen': 0.1, 'rem': 0.12500000000000003}, 'westBottom': {'gen': 0.1, 'rem': 0.12500000000000003}, 'eas

In [23]:
[x['cycle_length'] for x in list(data['O_rule_set_params'])]

TypeError: string indices must be integers

In [4]:
df1 = data.iloc[:20, :]
df2 = data.iloc[20:40, :]
df3 = data.iloc[40:60, :]

In [6]:
df1[df1['M_model_type'] == "deepmind"]['eval_avg_steps_taken'].mean()

5.454

In [7]:
df1[df1['M_model_type'] == "nervenet"]['eval_avg_steps_taken'].mean()

5.871999999999999

In [8]:
df1[df1['M_model_type'] == "fully_connected"]['eval_avg_steps_taken'].mean()

5.286

In [9]:
df1[df1['M_model_type'] == "no_structure"]['eval_avg_steps_taken'].mean()

4.99

In [10]:
df2[df2['M_model_type'] == "deepmind"]['eval_avg_steps_taken'].mean()

6.909999999999999

In [11]:
df2[df2['M_model_type'] == "nervenet"]['eval_avg_steps_taken'].mean()

6.774000000000001

In [12]:
df2[df2['M_model_type'] == "fully_connected"]['eval_avg_steps_taken'].mean()

6.764000000000001

In [13]:
df2[df2['M_model_type'] == "no_structure"]['eval_avg_steps_taken'].mean()

7.0120000000000005

In [14]:
df3[df3['M_model_type'] == "deepmind"]['eval_avg_steps_taken'].mean()

5.5200000000000005

In [15]:
df3[df3['M_model_type'] == "nervenet"]['eval_avg_steps_taken'].mean()

5.262

In [16]:
df3[df3['M_model_type'] == "fully_connected"]['eval_avg_steps_taken'].mean()

5.256

In [17]:
df3[df3['M_model_type'] == "no_structure"]['eval_avg_steps_taken'].mean()

4.396